# A simple model in `keras`

Notebook by [Aaron Berk](http://asberk.ca) for the 2017 [BC Data Science Workshop](http://workshop.bcdata.ca).

## Import packages

We'll need `numpy` to format the image data — we won't center the data, but we will map it to $[0,1]$ and reshape it.

We'll be building a dense network using `Dense` layers, and we'll train using [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). More information is available on [dropout](https://en.wikipedia.org/wiki/Dropout_(neural_networks)) and [`Dropout` layers](https://keras.io/layers/core/#dropout).

We'll again use the MNIST digits as our data set. This time, we can't load a one-hot encoding of the labels, so we'll have to transform it. We could use `sklearn`'s built-in method, but instead we'll keep it all in `keras` and use `to_categorical` (which is a more descriptive name, anyway). 

Lastly, we'll do something fancy and use a `callback`. `keras` callbacks allow you to do [many super cool things](https://www.tensorflow.org/get_started/summaries_and_tensorboard). For the purposes of this tutorial, we'll keep it simple and use a Callback to tell us when we've trained enough. 

In [ ]:
import keras 
from keras import backend as K

In [ ]:
import numpy as np

from keras.layers import Dense, Dropout, Input
from keras.models import Model

from keras.datasets import mnist
from keras.utils import to_categorical

from keras.callbacks import EarlyStopping

## Build the model

The model-building interface in `keras` is deliciously simple (it is not *quite* so simple when building models in `tensorflow`). For this network we'll be feeding the vectorized 28 x 28 images (*i.e.,* 784 in length) into the first layer of the model. The first layer of the model is a dense layer with 128 nodes, each performing their own matrix multiplications and [ReLU activations](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)). 

Next is Dropout: during the training phase, we'll random select a proportion of .25 of the weights in the first Dense layer and random reset them (with Gaussian noise, I think). From what I've learned, this has two important purposes:

1. it helps with overfitting
2. it helps to avoid the phenomenon where weights get caught in the same part of parameter space (which is kind of like 1.)

Next, we repeat the same thing as above, but use a Dropout proportion of .5. Lastly, we have a Dense layer of ten nodes whose activation is a [softmax function](https://en.wikipedia.org/wiki/Softmax_function), which is well suited to outputting a probability for each node, that we will interpret as *class membership*. 

In [ ]:
input_img = Input((784,))
model = Dense(128, activation='relu')(input_img)
model = Dropout(.25)(model)
model = Dense(128, activation='relu')(model)
model = Dropout(.5)(model)
model = Dense(10, activation='softmax')(model)

The next step in building the model is to call `keras`'s `Model` constructor with `input`, `output`, and [for fun] we'll give it a name. 

In [ ]:
digits_clf = Model(input_img, model, name='digits_clf')

Once the model has been constructed, we compile the model. This requires that we pass a [loss function](https://keras.io/losses/) (*e.g.,* [`categorical_crossentropy`](https://en.wikipedia.org/wiki/Cross_entropy), `rmsprop`, `mean_squared_error`, `hinge`, *etc.*), an [optimization method](https://keras.io/optimizers/) (*e.g.*, [`adagrad`](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf), `adadelta`, `adam`, `sgd`, *etc.*), and [optionally] some [metrics](https://keras.io/metrics/) to measure our progress. 

In [ ]:
digits_clf.compile('adagrad', 'categorical_crossentropy', metrics=['mse', 'accuracy'])

## Load the data

We'll be slightly blasphemous and use the *test* data as the *validation* data. As an exercise and to keep machine learning canon alive, re-write this section to split the training data into a training set and a development/validation set. 

In [ ]:
(x_train, y_train), (_, _) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
print()
print(x_train.shape)

The `y` data happens to appear numerical, but *should* be categorical. We can use `keras.utils.to_categorical` to do "one-hot" encoding, getting a format more preferable for machine learning classification. 

In [ ]:
y_train_1h = to_categorical(y_train)

## Training phase

The next step is to `fit` the model to the data, using `(x_test, y_test_1h)` as the validation set. For at most "30 passes", the algorithm will feed a batch of `256` images at a time into the model and use the `adagrad` optimizer to "step in the right direction" (à la Gradient Descent). We're shuffling the data in case it was in any way ordered. Lastly, we'll use a [`callback`](https://keras.io/callbacks/#earlystopping) function to halt the training early if we have achieved sufficient convergence. See the link above (or the documentation by pressing `Shift`+`Tab`) to learn more about `EarlyStopping` and its `patience` argument. 

In [ ]:
digits_clf.fit(x_train, y_train_1h, epochs=30, batch_size=256, 
               shuffle=True, validation_split=.25,
               callbacks=[EarlyStopping(patience=3)])

## Save the model

It will be useful to save and load our models — that way if we want to exit Python we don't lose all of our progress! So let's do that and pick things up in the next notebook. :-)

In [ ]:
digits_clf.save('./simple-digits_clf.h5')